### Sample one image from the dataset

In [1]:
import torch
from detection.pennfudan_dataset import PennFudanDataset, get_transform

# use our dataset and defined transformations
dataset = PennFudanDataset('./PennFudanPed', get_transform(train=True))
dataset_test = PennFudanDataset('./PennFudanPed', get_transform(train=False))

# split the dataset in train and test set
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:-50])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

In [2]:
img,target = dataset_test[0]
dataset_test[0]

(tensor([[[0.3451, 0.3255, 0.3216,  ..., 0.5137, 0.6196, 0.5255],
          [0.3333, 0.3216, 0.3294,  ..., 0.4471, 0.6157, 0.5961],
          [0.3216, 0.3216, 0.3333,  ..., 0.3961, 0.5765, 0.6039],
          ...,
          [0.7725, 0.7843, 0.7725,  ..., 0.7098, 0.7176, 0.7255],
          [0.6706, 0.6824, 0.7059,  ..., 0.7059, 0.7137, 0.7255],
          [0.7137, 0.7373, 0.7333,  ..., 0.7216, 0.7216, 0.7176]],
 
         [[0.4039, 0.3843, 0.3765,  ..., 0.5843, 0.7020, 0.6118],
          [0.3922, 0.3804, 0.3843,  ..., 0.5137, 0.6980, 0.6784],
          [0.3804, 0.3804, 0.3882,  ..., 0.4627, 0.6549, 0.6863],
          ...,
          [0.7529, 0.7647, 0.7529,  ..., 0.6824, 0.6902, 0.6980],
          [0.6510, 0.6627, 0.6863,  ..., 0.6784, 0.6863, 0.6980],
          [0.6941, 0.7176, 0.7137,  ..., 0.6941, 0.6941, 0.6902]],
 
         [[0.4863, 0.4588, 0.4275,  ..., 0.4980, 0.6353, 0.5647],
          [0.4745, 0.4549, 0.4353,  ..., 0.4392, 0.6392, 0.6431],
          [0.4627, 0.4549, 0.4392,  ...,

### Run the explainer and output the class probability

In [3]:
import warnings 
warnings.filterwarnings('ignore')

In [4]:
from src.sodexplainer import SODExplainer
explainer = SODExplainer(load_from='./checkpoints/faster_rcnn_10_epochs.ckpt')
prob = explainer.get_class_probability(dataset_test[0])
prob

tensor([0.9987, 0.0013])

### Check output 
#### Compute IoUs

In [5]:
from src.utils import jaccard
prediction = explainer.model(img.unsqueeze(0))
ious = jaccard(prediction[0]["boxes"],target['boxes'])
ious

tensor([[0.9555],
        [0.0650]], grad_fn=<DivBackward0>)

#### Get argmax 

In [6]:
import numpy as np
obj_idx,box_idx = np.unravel_index(torch.argmax(ious), ious.shape)

In [7]:
prediction[0]

{'boxes': tensor([[278.4743,  91.1026, 398.2475, 369.9154],
         [260.3747, 207.0027, 296.9782, 347.9638]], grad_fn=<StackBackward0>),
 'labels': tensor([1, 1]),
 'scores': tensor([0.9987, 0.1899], grad_fn=<IndexBackward0>)}

In [8]:
prediction[0]['scores'][box_idx]

tensor(0.9987, grad_fn=<SelectBackward0>)

The score corresponds to the output probability. Done.